In [41]:
import pandas as pd
import numpy as np
import re
import nltk
import urllib.request
import os
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 90.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00


In [42]:
# Load review data from CSV
trip_df = pd.read_excel("/content/TripAdvisor.xlsx",sheet_name = "in")


#feature 1 : extracted no of helpful votes made by each user from contribution column

In [43]:
trip_df['Contributions'].fillna('',inplace = True)
trip_df['Helpulness'] = trip_df['Contributions'].apply(lambda x:re.search('(\d+)(\s*)helpful', x, re.IGNORECASE))
trip_df['Helpulness'] = trip_df['Helpulness'].apply(lambda x:int(x.group(1)) if x is not None else 0)
trip_df.head()
# Convert DataFrame to Excel
#trip_df.to_excel('trip_data.xlsx', index=False)

,User_Name,Contributions,Title,Review,Date_of_Stay,Heading,Helpulness
0,T5329NXmaryh,"Louisville, Kentucky;3 contributions",['Shantel & Luigi welcomed us with smiles and ...,Shantel & Luigi and all front desk staff are e...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0
1,jcmtl1965,"Montreal, Canada;8938 contributions;8938;100 h...",['Nice stay for my return to New York'],I hadn't been to New York in 22 months (which ...,Date of stay: August 2021,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,100
2,Venture02710567543,1 contribution,['Perfect stay with perfect view'],The convenience of checking in with Nakita Wan...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0
3,Leigh H,1 contribution,['Service with a smile'],"The front desk staff at our check-in, Nakita, ...",Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0
4,GrandTour48028656883,1 contribution,['Highly recommend!'],Everyone at this hotel was great! Jaquanna hel...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0


#feature 2: extracted no of contributions made by each user from contribution column

In [44]:
# Extract the number of contributions
trip_df['Num_Contributions'] = trip_df['Contributions'].apply(lambda x: int(re.findall(r'(\d+)', x)[0]) if re.findall(r'(\d+)', x) else 0)
trip_df.head()

,User_Name,Contributions,Title,Review,Date_of_Stay,Heading,Helpulness,Num_Contributions
0,T5329NXmaryh,"Louisville, Kentucky;3 contributions",['Shantel & Luigi welcomed us with smiles and ...,Shantel & Luigi and all front desk staff are e...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,3
1,jcmtl1965,"Montreal, Canada;8938 contributions;8938;100 h...",['Nice stay for my return to New York'],I hadn't been to New York in 22 months (which ...,Date of stay: August 2021,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,100,8938
2,Venture02710567543,1 contribution,['Perfect stay with perfect view'],The convenience of checking in with Nakita Wan...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,1
3,Leigh H,1 contribution,['Service with a smile'],"The front desk staff at our check-in, Nakita, ...",Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,1
4,GrandTour48028656883,1 contribution,['Highly recommend!'],Everyone at this hotel was great! Jaquanna hel...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,1


#feature 3: length of review made by each user

In [45]:
# Perform text preprocessing
stopwords_list = stopwords.words('english')

# Convert text to lowercase, remove stopwords and punctuation
trip_df['Clean_Review'] = trip_df['Review'].str.lower()
trip_df['Clean_Review'] = trip_df['Clean_Review'].astype(str)
trip_df['Clean_Review'] = trip_df['Clean_Review'].apply(lambda x: ' '.join([word for word in nltk.word_tokenize(str(x)) if word.isalnum() and word not in stopwords_list]))
# Use TF-IDF for numerical representation
#tfidf = TfidfVectorizer()
#tfidf_features = tfidf.fit_transform(trip_df['Clean_Review'])

# Extract length of the review as a feature
trip_df['Review_Length'] = trip_df['Clean_Review'].apply(lambda x: len(str(x)) if not pd.isnull(x) else 0)
trip_df.head()
#trip_df.to_excel('trip_data.xlsx', index=False)

,User_Name,Contributions,Title,Review,Date_of_Stay,Heading,Helpulness,Num_Contributions,Clean_Review,Review_Length
0,T5329NXmaryh,"Louisville, Kentucky;3 contributions",['Shantel & Luigi welcomed us with smiles and ...,Shantel & Luigi and all front desk staff are e...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,3,shantel luigi front desk staff excellent room ...,168
1,jcmtl1965,"Montreal, Canada;8938 contributions;8938;100 h...",['Nice stay for my return to New York'],I hadn't been to New York in 22 months (which ...,Date of stay: August 2021,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,100,8938,new york 22 months really long decided stay aw...,289
2,Venture02710567543,1 contribution,['Perfect stay with perfect view'],The convenience of checking in with Nakita Wan...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,1,convenience checking nakita wanda absolute eas...,181
3,Leigh H,1 contribution,['Service with a smile'],"The front desk staff at our check-in, Nakita, ...",Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,1,front desk staff nakita wanda luigi stellar fr...,324
4,GrandTour48028656883,1 contribution,['Highly recommend!'],Everyone at this hotel was great! Jaquanna hel...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,1,everyone hotel great jaquanna helped get us ro...,205


#featur 4 : extract city, state/country of user from contributions column

In [46]:
# Extract city, state, or country from 'Contributions'
# Split 'Contributions' column into temporary columns
split_columns = trip_df['Contributions'].str.split(',', expand=True)

# Extract city, state, or country from temporary columns
trip_df['City'] = split_columns[0].str.strip()
trip_df['State/Country'] = split_columns[1].str.strip().str.split(';', expand=True)[0]
no_comma_mask = split_columns[1].isna()
trip_df.loc[no_comma_mask, ['City', 'State/Country']] = None
trip_df.head()

,User_Name,Contributions,Title,Review,Date_of_Stay,Heading,Helpulness,Num_Contributions,Clean_Review,Review_Length,City,State/Country
0,T5329NXmaryh,"Louisville, Kentucky;3 contributions",['Shantel & Luigi welcomed us with smiles and ...,Shantel & Luigi and all front desk staff are e...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,3,shantel luigi front desk staff excellent room ...,168,Louisville,Kentucky
1,jcmtl1965,"Montreal, Canada;8938 contributions;8938;100 h...",['Nice stay for my return to New York'],I hadn't been to New York in 22 months (which ...,Date of stay: August 2021,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,100,8938,new york 22 months really long decided stay aw...,289,Montreal,Canada
2,Venture02710567543,1 contribution,['Perfect stay with perfect view'],The convenience of checking in with Nakita Wan...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,1,convenience checking nakita wanda absolute eas...,181,None,None
3,Leigh H,1 contribution,['Service with a smile'],"The front desk staff at our check-in, Nakita, ...",Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,1,front desk staff nakita wanda luigi stellar fr...,324,None,None
4,GrandTour48028656883,1 contribution,['Highly recommend!'],Everyone at this hotel was great! Jaquanna hel...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,1,everyone hotel great jaquanna helped get us ro...,205,None,None


#feature 5: extract popular features of stay from title column and the frequency of words used

In [47]:

# Remove stop words from the words list
stopwords_list = set(stopwords.words('english'))
stopwords_list.add('the')
# Create a new column 'Popular_Features'
trip_df['Popular_Features'] = ""

# Iterate over each user and find the top 3 features mentioned in their titles
for user in trip_df['User_Name'].unique():
    user_titles = trip_df[trip_df['User_Name'] == user]['Title'].astype(str)
    user_words = [word for title in user_titles for word in title.lower().split() if word.lower() not in stopwords_list]
    user_words = [re.sub(r'[^\w\s]', '', word) for word in user_words]
    user_word_freq = Counter(user_words)
    top_features = [word for word, count in user_word_freq.most_common(3)]
    trip_df.loc[trip_df['User_Name'] == user, 'Popular_Features'] = ', '.join(top_features)

trip_df.head()


,User_Name,Contributions,Title,Review,Date_of_Stay,Heading,Helpulness,Num_Contributions,Clean_Review,Review_Length,City,State/Country,Popular_Features
0,T5329NXmaryh,"Louisville, Kentucky;3 contributions",['Shantel & Luigi welcomed us with smiles and ...,Shantel & Luigi and all front desk staff are e...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,3,shantel luigi front desk staff excellent room ...,168,Louisville,Kentucky,"us, shantel,"
1,jcmtl1965,"Montreal, Canada;8938 contributions;8938;100 h...",['Nice stay for my return to New York'],I hadn't been to New York in 22 months (which ...,Date of stay: August 2021,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,100,8938,new york 22 months really long decided stay aw...,289,Montreal,Canada,"nice, stay, return"
2,Venture02710567543,1 contribution,['Perfect stay with perfect view'],The convenience of checking in with Nakita Wan...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,1,convenience checking nakita wanda absolute eas...,181,None,None,"perfect, stay, view"
3,Leigh H,1 contribution,['Service with a smile'],"The front desk staff at our check-in, Nakita, ...",Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,1,front desk staff nakita wanda luigi stellar fr...,324,None,None,"service, smile"
4,GrandTour48028656883,1 contribution,['Highly recommend!'],Everyone at this hotel was great! Jaquanna hel...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,1,everyone hotel great jaquanna helped get us ro...,205,None,None,"highly, recommend"


#feature 6: classfiy the cleaned review column and get the sentiment

In [48]:
from nltk.sentiment import SentimentIntensityAnalyzer
#valence aware dictionary and sentiment reasoner - pretraine model
#nltk.download('vader_lexicon')
# Initialize the sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Apply sentiment analysis to the cleaned review column
trip_df['Sentiment'] = trip_df['Clean_Review'].apply(lambda x: sid.polarity_scores(x)['compound'])

# Classify the sentiment as positive, negative, or neutral based on the compound score
trip_df['Review_Sentiment_Class'] = trip_df['Sentiment'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))
trip_df.head()


#feature 7: extract season of the stay of user from date of stay column to identify seasonal patterns or preferences

In [51]:
# Create a mapping of month to season
season_mapping = {
    1: 'Winter',
    2: 'Spring',
    3: 'Spring',
    4: 'Spring',
    5: 'Summer',
    6: 'Summer',
    7: 'Summer',
    8: 'Autumn',
    9: 'Autumn',
    10: 'Autumn',
    11: 'Winter',
    12: 'Winter'
}

trip_df['Date_of_Stay'] = trip_df['Date_of_Stay'].str.replace('Date of stay: ', '')
# Extract the month from the 'Date_of_Stay' column
trip_df['Month'] = pd.to_datetime(trip_df['Date_of_Stay']).dt.month

# Map the month to the corresponding season
trip_df['Season'] = trip_df['Month'].map(season_mapping)

trip_df.head()
trip_df.to_excel('trip_data.xlsx', index=False)

#not required code

In [7]:
trip_df['Title+Review'] = trip_df['Title'] + ' ' + trip_df['Review']
trip_df['Clean_Review'] = trip_df['Title+Review'].str.lower()
trip_df['Clean_Review'] = trip_df['Clean_Review'].str.replace('[^a-z0-9]+',' ',regex=True)
trip_df['Clean_Review'] = trip_df['Clean_Review'].astype(str)
#trip_df['Clean_Title'] = trip_df['Clean_Title'].apply(lambda x:str(TextBlob(x).correct()))
trip_df['Clean_Review'] = trip_df['Clean_Review'].apply(lambda x: word_tokenize(x))
stopwords_list = stopwords.words('english')
trip_df['Clean_Review'] = trip_df['Clean_Review'].apply(lambda x: [word for word in x if word not in stopwords_list])
wordnet_lemmatizer = WordNetLemmatizer()
trip_df['Clean_Review'] = trip_df['Clean_Review'].apply(lambda x:[wordnet_lemmatizer.lemmatize(word) for word in x])
trip_df = trip_df[trip_df['Clean_Review'].str.len() > 1]
trip_df.head()

,User_Name,Contributions,Title,Review,Date_of_Stay,Heading,Helpulness,Title+Review,Clean_Review
0,T5329NXmaryh,"Louisville, Kentucky;3 contributions",['Shantel & Luigi welcomed us with smiles and ...,Shantel & Luigi and all front desk staff are e...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,['Shantel & Luigi welcomed us with smiles and ...,"[shantel, luigi, welcomed, u, smile, informati..."
1,jcmtl1965,"Montreal, Canada;8938 contributions;8938;100 h...",['Nice stay for my return to New York'],I hadn't been to New York in 22 months (which ...,Date of stay: August 2021,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,100,['Nice stay for my return to New York'] I hadn...,"[nice, stay, return, new, york, new, york, 22,..."
2,Venture02710567543,1 contribution,['Perfect stay with perfect view'],The convenience of checking in with Nakita Wan...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,['Perfect stay with perfect view'] The conveni...,"[perfect, stay, perfect, view, convenience, ch..."
3,Leigh H,1 contribution,['Service with a smile'],"The front desk staff at our check-in, Nakita, ...",Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,['Service with a smile'] The front desk staff ...,"[service, smile, front, desk, staff, check, na..."
4,GrandTour48028656883,1 contribution,['Highly recommend!'],Everyone at this hotel was great! Jaquanna hel...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,['Highly recommend!'] Everyone at this hotel w...,"[highly, recommend, everyone, hotel, great, ja..."


In [8]:
from transformers import BertTokenizer, BertModel

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
# Extract the review texts from the 'Review' column
reviews = trip_df['Review'].tolist()
encoded_reviews = tokenizer(reviews, padding=True, truncation=True, return_tensors='pt')

In [9]:
import torch
from transformers import BertTokenizer, BertModel

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Assuming you have already encoded the reviews as 'encoded_reviews'

# Reduce batch size
batch_size = 16  # Adjust the batch size as needed

# Limit the number of reviews
max_reviews = 1000  # Adjust the maximum number of reviews as needed

# Process reviews in chunks or batches
for i in range(0, len(encoded_reviews['input_ids']), batch_size):
    batch_reviews = {key: value[i:i+batch_size] for key, value in encoded_reviews.items()}
    if i >= max_reviews:
        break

    with torch.no_grad():
        outputs = model(**batch_reviews)

    # Extract features from a specific layer (e.g., second-to-last layer)
    features = outputs.last_hidden_state[:, 0, :]

    # Process the extracted features as needed
    # ...



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: ignored

In [2]:
import torch
torch.cuda.empty_cache()